In [1]:
import pandas as pd
import numpy as np
import os
import string
import datetime as dt
import pathlib

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
from pattern.text.en import singularize

In [2]:
def create_document_list(path):
    wd = os.getcwd()
    file_list = os.listdir()
    return(file_list)

def read_text_file(file_path):
	with open(file_path, 'r', encoding='utf-8' ) as f:
	# with open(file_path, 'r', encoding='cp1252' ) as f:
		return f.read()
		
def print_punctuation(input_string=None):
	punct_str = ""
	for i in range(len(input_string)):
		char = input_string[i] 
		char = char.replace('”', "\"")
		char = char.replace('“', "\"")
		char = char.replace("\n", " ")
		if char in string.punctuation:
			punct_str = punct_str + char
	return punct_str

def text_lowercase(text):
    return text.lower()

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)    

def remove_whitespace(text):
    return  " ".join(text.split())

def count_sentences(text):
	return len(sent_tokenize(text))

def count_words(text):
    return len(word_tokenize(text))
    # word_count = [n+1 for word in word_tokens]

def count_distinct_words(text):
    return len(set(word_tokenize(text)))

def build_vocab(existing_list, new_text, distinct=True):
    new_words = word_tokenize(new_text)
    existing_list.extend(new_words)
    if distinct==True:
        return list(set(existing_list))
    else:
        return(list(existing_list))

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

def clean_string(text):
    text = text.replace("’s", "")
    text = text.replace("'s", "")
    text = text.replace('”', "\"")
    text = text.replace('“', "\"")
    text = text.replace('’', "\'")
    # adding this to split hyptenated words into separate words
    text = text.replace('-', " ")
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    text = ' '.join([str(elem) for elem in text])
    return text	

def clean_incl_stopwords(text):
    text = text.replace('”', "\"")
    text = text.replace('“', "\"")
    text = text.replace('’', "\'")
    # adding this to split hyptenated words into separate words
    text = text.replace('-', " ")
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    # text = remove_stopwords(text)
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens]
    text = ' '.join([str(elem) for elem in filtered_text])
    return text	    

def vocab_freq(word_list,
               _singularize=False,
               top_n=None):

    _dict = {
        'word':[],
        'freq':[]
    }

    if _singularize == True:
        singles = [singularize(plural) for plural in word_list]
        distinct = list(set(singles))
        for d in range(len(distinct)):
            word = distinct[d]
            freq = singles.count(word)
            _dict['word'].append(word)
            _dict['freq'].append(freq)
    else:
        distinct = list(set(word_list))
        for d in range(len(distinct)):
            word = distinct[d]
            freq = word_list.count(word)
            _dict['word'].append(word)
            _dict['freq'].append(freq)

    df = pd.DataFrame(_dict)
    df.sort_values(by='freq', ascending=False, inplace=True)
    df.set_index('word', inplace=True)
    
    if top_n:
        return df.head(top_n)
    else:
        return df

def stack_string(text, max_length=25):
    stacked = ''
    n = 0
    for i in range(len(text)):
        stacked = stacked + text[i]
        n+=1
        if n == max_length:
            stacked = stacked + '\n'
            n = 0
    return stacked + '\n'    

def count_quoted_words(text):
    text = text.replace('”', "\"")
    text = text.replace('“', "\"")
    quoted_words = ''
    start_quote = 0
    mid_quote = 0
    end_quote = 0
    for char in range(len(text)):
        if start_quote == 1 and end_quote == 1:
            start_quote = 0
            end_quote = 0

        if text[char] == "\"" and start_quote == 0 and end_quote == 0:
            start_quote = 1

        if text[char] != "\"" and start_quote == 1 and end_quote == 0:
            mid_quote = 1
            quoted_words += text[char]

        if text[char] == "\"" and start_quote == 1 and mid_quote == 1:
            quoted_words += " "
            mid_quote = 0
            end_quote = 1

    quoted_words = clean_incl_stopwords(quoted_words)
    word_count = count_words(quoted_words)

    return word_count

def stats_quoted_words(text):
    text = text.replace('”', "\"")
    text = text.replace('“', "\"")
    quoted_words = ''
    start_quote = 0
    mid_quote = 0
    end_quote = 0
    short_quotes = []
    long_quotes = []
    quote_list = []
    quote_word_len_list = []
    
    dict = {
        'quote_word_count':[],
        'quote_count':[],
        'count_short':[],
        'count_long':[],
        'avg_len_short':[],
        'avg_len_long':[]
    }

    for char in range(len(text)):
        if start_quote == 1 and end_quote == 1:
            quoted_words = ''
            start_quote = 0
            end_quote = 0

        if text[char] == "\"" and start_quote == 0 and end_quote == 0:
            start_quote = 1

        if text[char] != "\"" and start_quote == 1 and end_quote == 0:
            mid_quote = 1
            quoted_words += text[char]

        if text[char] == "\"" and start_quote == 1 and mid_quote == 1:
            quoted_words = clean_incl_stopwords(quoted_words)
            quote_list.append(quoted_words)
            quote_word_len_list.append(count_words(quoted_words))

            if count_words(quoted_words) > 0 and count_words(quoted_words) <= 3:
                short_quotes.append(count_words(quoted_words))

            if count_words(quoted_words) > 3:
                long_quotes.append(count_words(quoted_words))
            
            mid_quote = 0
            end_quote = 1

    dict['quote_word_count'].append(sum(quote_word_len_list))
    dict['quote_count'].append(len(quote_word_len_list))
    dict['count_short'].append(len(short_quotes))
    dict['count_long'].append(len(long_quotes))

    if len(short_quotes) > 0:
        dict['avg_len_short'].append(round(sum(short_quotes) / len(short_quotes),1))
    else:
        dict['avg_len_short'].append(0)

    if len(long_quotes) > 0:
        dict['avg_len_long'].append(round(sum(long_quotes) / len(long_quotes), 1))
    else:
        dict['avg_len_long'].append(0)

    return pd.DataFrame(dict)    

In [77]:
# getting directory path of top of repository
dir_top = os.getcwd()
columnist = 'Mike Hulett'

# file names are in YYYYMMDD.txt format
format = '%Y%m%d'
columnist_files = os.listdir(os.chdir(dir_top + './' + columnist))

dict = {
	"columnist":[],
	"file_date":[],
	"story_punctuation":[],
	"word_count":[],
	"distinct_word_count":[],
	'quote_word_count':[],
	"sentence_count":[],
	'quote_count':[],
	'count_short_quote':[],
	'count_long_quote':[],
	'avg_len_short_quote':[],
	'avg_len_long_quote':[],
	'all_words':[]
}

dict_vocab = {
	"columnist":[],
	"distinct_words":[],
	"distinct_words_excl_stop":[],
	"all_nonstop_words":[]
}

dict_sentences = {
	"columnist":[],
	"file_date":[],
	"text":[]
}

vocab_w_stop = []
vocab_no_stop = []
all_words = []
columnist_sentences = []
list_bigrams = []
list_trigrams = []

# iterate through all files
for i in range(len(columnist_files)):
	file = columnist_files[i]
	ymd = file[:8]

	# Check whether file is in text format or not
	if file.endswith(".txt"):
		# file_path = f"{wd}\{file}"
		file_date = dt.datetime.strptime(ymd, format)
		s = read_text_file(file)
		cleaned_string = clean_string(s)
		clean_with_stopwords = clean_incl_stopwords(s)
		p = print_punctuation(s)
		vocab_no_stop = build_vocab(vocab_no_stop, cleaned_string)
		vocab_w_stop = build_vocab(vocab_w_stop, clean_with_stopwords)
		all_words = build_vocab(all_words, cleaned_string, distinct=False)

		words = nltk.word_tokenize(cleaned_string)
		list_bigrams.extend(list(nltk.bigrams(words)))
		list_trigrams.extend(list(nltk.trigrams(words)))
		
		columnist_sentences = sent_tokenize(s)
		for i in range(len(columnist_sentences)):
			dict_sentences["columnist"].append(columnist)
			dict_sentences["file_date"].append(file_date)
			dict_sentences["text"].append(text_lowercase(columnist_sentences[i]))

		dx = stats_quoted_words(s)

		dict["columnist"].append(columnist)
		dict["file_date"].append(file_date)
		dict["story_punctuation"].append(p)
		dict["word_count"].append(count_words(s))
		dict["distinct_word_count"].append(count_distinct_words(s))
		dict["sentence_count"].append(count_sentences(s))
		dict["quote_word_count"].append(dx.quote_word_count[0])
		dict["quote_count"].append(dx.quote_count[0])
		dict["count_short_quote"].append(dx.count_short[0])
		dict["count_long_quote"].append(dx.count_long[0])
		dict['avg_len_short_quote'].append(dx.avg_len_short[0])
		dict["avg_len_long_quote"].append(dx.avg_len_long[0])
		dict["all_words"].append(cleaned_string)

dict_vocab["columnist"].append(columnist)
dict_vocab["distinct_words"].append(vocab_w_stop)
dict_vocab["distinct_words_excl_stop"].append(vocab_no_stop)
dict_vocab["all_nonstop_words"].append(all_words)

df = pd.DataFrame(dict)
df_vocab = pd.DataFrame(dict_vocab)
df_sentences = pd.DataFrame(dict_sentences)

In [110]:
df

,columnist,file_date,story_punctuation,word_count,distinct_word_count,quote_word_count,sentence_count,quote_count,count_short_quote,count_long_quote,avg_len_short_quote,avg_len_long_quote,all_words
0,Mike Hulett,2020-04-11,",."","".,-,.,.,-.,,',-.,-.,,-..,.,,.,,""."",.""."",,...",631,374,71,28,7,3,4,2.0,16.2,wuhan virus dominates presidential election lo...
1,Mike Hulett,2020-04-18,",:"",,.,..""!..,,,.:"",,."".,.:""(),."".:"",,--?""?:"",...",607,321,176,29,10,3,7,1.7,24.4,april th column jim shaw wrote last two weeks ...
2,Mike Hulett,2020-05-09,";,..!,./.,,.%..:,,,,,,;.,,,..,....!.,....,,.,....",585,314,0,33,0,0,0,0.0,0.0,dollar every forum editorial disagreed years w...
3,Mike Hulett,2020-05-23,"-.,,....,-.,,-.,.-.-."",""-?.-"""".-"""",-.."".""-,""""....",616,361,36,30,9,5,4,1.2,7.5,declassified justice department documents conf...
4,Mike Hulett,2020-06-20,".,??..,:,,,-?:"",,.""??,?,.""""."""".,,.,,.,.,,-..:,...",591,303,19,31,5,4,1,2.0,11.0,moorhead mayor jonathan judd sensibly proposed...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Mike Hulett,2022-10-08,"""?"",.,.,-.--..-"","","","".-:"".."",!,:""..,,...."":""?...",587,337,181,20,8,0,8,0.0,22.6,opinion website featured bizarre headline qano...
63,Mike Hulett,2022-10-22,",.,.-,,,""."",,.,.,,%,"""".%.""-,.""."";."".""."","""".,-?...",572,320,142,18,8,3,5,3.0,26.6,former speaker house newt gingrich highly resp...
64,Mike Hulett,2022-11-05,",:,-,.,.,,,-.""-"",.,.-."""".!,.,--,-,,--,.-,,..-....",583,349,14,27,5,5,0,2.8,0.0,moderate democrat independent republican reade...
65,Mike Hulett,2022-11-19,".,.,,.-(!),..-.,,.:"",-,."",..""?""(-:,,,,,-,!)""-""...",589,336,85,21,10,2,8,2.0,10.1,focusing issues every opinion columnist expect...


In [121]:
wc = df.groupby(['columnist'])['word_count'].mean().reset_index()
lc = df.groupby(['columnist'])['avg_len_long_quote'].mean().reset_index()
sq = df.groupby(['columnist'])['count_short_quote'].mean().reset_index()
lq = df.groupby(['columnist'])['count_long_quote'].mean().reset_index()

sq = df[df.count_long_quote > 0]
sq = list(df.sentence_count.to_list())

print(np.average(sq))


26.223880597014926


In [113]:
print(wc, lc, sq, lq)

     columnist  word_count
0  Mike Hulett  590.432836      columnist  avg_len_long_quote
0  Mike Hulett           15.132836      columnist  count_short_quote
0  Mike Hulett           4.761194      columnist  count_long_quote
0  Mike Hulett          2.925373


In [101]:
distinct_words = df_vocab.distinct_words_excl_stop.iloc[0]
all_words = df_vocab.all_nonstop_words.iloc[0]

dict_words = {
    'word':[],
    'count':[]
}

for i in range(len(words)):
    w = words[i]
    dict_words['word'].append(w)
    dict_words['count'].append(all_words.count(w))

df_words = pd.DataFrame(dict_words)

unique_bigrams = list(set(list_bigrams))
unique_trigrams = list(set(list_trigrams))

dict_bigrams = {
    'bigram':[],
    'count':[]
}

dict_trigrams = {
    'trigram':[],
    'count':[]
}

for i in range(len(unique_bigrams)):
    b = unique_bigrams[i]
    c = list_bigrams.count(b)
    dict_bigrams['bigram'].append(b)
    dict_bigrams['count'].append(c)

for i in range(len(unique_trigrams)):
    b = unique_trigrams[i]
    c = list_trigrams.count(b)
    dict_trigrams['trigram'].append(b)
    dict_trigrams['count'].append(c)

df_bigrams = pd.DataFrame(dict_bigrams)
df_trigrams = pd.DataFrame(dict_trigrams)    



In [ ]:
# def count_docs_contain_term(string=None):
#     count = 0
#     for i in range(len(columnist_files)):
#         file = columnist_files[i]
#         s = read_text_file(file)
#         cleaned_string = clean_string(s)
#         if cleaned_string.count(term) > 0:
#             count += 1
#     return count

# def inverse_doc_freq(term):
#     for i in range(len(columnist_files)):
#         file = columnist_files[i]
#         s = read_text_file(file)
#         cleaned_string = clean_string(s)
#         if cleaned_string.count(term) > 0:
#             document_contains_term += 1
#     return np.log(len(columnist_files) / document_contains_term)  

In [85]:
df_trigrams.sort_values(by='count', ascending=False).head(10)

,trigram,count
16935,"(black, lives, matter)",21
13221,"(president, joe, biden)",13
13297,"(president, donald, trump)",9
10130,"(former, president, donald)",9
5147,"(border, patrol, agents)",7
20037,"(speaker, nancy, pelosi)",7
15571,"(critical, race, theory)",7
5692,"(fargo, moorhead, west)",7
5507,"(moorhead, west, fargo)",7
12181,"(vice, president, kamala)",6


In [102]:
blm_sentences =  df_sentences[df_sentences['text'].str.contains('black lives matter')]
blm_sentences

,columnist,file_date,text
130,Mike Hulett,2020-06-20,black lives matter is aggressively promoting i...
132,Mike Hulett,2020-06-20,"mainstream media have shown white protesters, ..."
212,Mike Hulett,2020-07-18,"“systemic police racism,” a volatile black liv..."
240,Mike Hulett,2020-08-01,"today, many well-meaning americans, peacefully..."
281,Mike Hulett,2020-08-29,"like clockwork, a black lives matter activist ..."
284,Mike Hulett,2020-08-29,"understand, when employed as a marxist black l..."
288,Mike Hulett,2020-08-29,"nevertheless, straight from the black lives ma..."
291,Mike Hulett,2020-08-29,"when re-elected, president trump will continue..."
375,Mike Hulett,2020-10-23,"danz repeated other unsubstantiated, slanderou..."
380,Mike Hulett,2020-10-23,how did danz dream up “the militia he’s unleas...


In [103]:
marxist_sentences =  df_sentences[df_sentences['text'].str.contains('marx')]
marxist_sentences

,columnist,file_date,text
159,Mike Hulett,2020-07-04,organized marxist anarchists have spread out a...
168,Mike Hulett,2020-07-04,armed marxist militias are tearing down artifa...
170,Mike Hulett,2020-07-04,clearly intended to expunge important chronicl...
172,Mike Hulett,2020-07-04,"sadly, liberals are unwittingly collaborating ..."
177,Mike Hulett,2020-07-04,"or, conversely, will america be judged by vote..."
240,Mike Hulett,2020-08-01,"today, many well-meaning americans, peacefully..."
242,Mike Hulett,2020-08-01,mainstream media describe nightly marxist prot...
243,Mike Hulett,2020-08-01,intentional marxist attacks on united states f...
284,Mike Hulett,2020-08-29,"understand, when employed as a marxist black l..."
291,Mike Hulett,2020-08-29,"when re-elected, president trump will continue..."


In [ ]:
document_contains_term = 0
term = 'black lives matter'
# corpus_length = 0
  
term_docs = count_docs_contain_term(term)

for i in range(len(columnist_files)):
    file = columnist_files[i]
    s = read_text_file(file)
    cleaned_string = clean_string(s)
    td_idf = cleaned_string.count(term) / count_words(s) * term_docs
    print('document', i, 'tdidf', td_idf)

In [104]:
df_bigrams.sort_values(by='count', ascending=False).head(10)

,bigram,count
13903,"(president, trump)",33
2863,"(joe, biden)",32
7588,"(mainstream, media)",30
8531,"(vice, president)",23
16713,"(black, lives)",22
7750,"(white, house)",21
15733,"(lives, matter)",21
3567,"(donald, trump)",19
16557,"(law, enforcement)",19
18904,"(border, patrol)",18


In [106]:
media_sentences = df_sentences[df_sentences['text'].str.contains('mainstream media')]
media_sentences

,columnist,file_date,text
148,Mike Hulett,2020-06-20,personal note: i have known hundreds of dedica...
707,Mike Hulett,2021-04-24,our border patrol van turned onto south sasabe...
716,Mike Hulett,2021-04-24,our border patrol instructors described how ca...
718,Mike Hulett,2021-04-24,"during scorching hot summer months, border pat..."
734,Mike Hulett,2021-04-24,just as tucson police officers under magnus re...
1020,Mike Hulett,2021-10-09,vice president kamala harris was “outraged” ov...
1022,Mike Hulett,2021-10-09,president joe biden angrily declared border pa...
1023,Mike Hulett,2021-10-09,"the truth is, border patrol agents don’t carry..."
1025,Mike Hulett,2021-10-09,"in the border patrol citizen academy, we learn..."
1027,Mike Hulett,2021-10-09,"the border patrol's horseback ""samaritans"" wou..."


In [79]:
df_words.sort_values(by='count', ascending=False).head(25)

document 0 tdidf 0.058637083993660855
document 1 tdidf 0.0
document 2 tdidf 0.0
document 3 tdidf 0.7808441558441559
document 4 tdidf 0.0
document 5 tdidf 0.061872909698996656
document 6 tdidf 0.0
document 7 tdidf 0.0
document 8 tdidf 0.49087893864013266
document 9 tdidf 0.25170068027210885
document 10 tdidf 0.06379310344827586
document 11 tdidf 0.0
document 12 tdidf 0.8946459412780657
document 13 tdidf 0.7474747474747475
document 14 tdidf 0.06357388316151202
document 15 tdidf 0.5522388059701493
document 16 tdidf 0.30833333333333335
document 17 tdidf 0.12171052631578946
document 18 tdidf 0.30377668308702793
document 19 tdidf 0.13167259786476868
document 20 tdidf 0.36333878887070375
document 21 tdidf 1.2993311036789297
document 22 tdidf 0.0
document 23 tdidf 0.18813559322033896
document 24 tdidf 0.0
document 25 tdidf 0.0
document 26 tdidf 0.06468531468531469
document 27 tdidf 0.0
document 28 tdidf 0.0
document 29 tdidf 0.0
document 30 tdidf 0.3130287648054145
document 31 tdidf 0.43676222

In [16]:
biden_sentences =  df_sentences[df_sentences['text'].str.contains('biden')]

In [17]:
biden_sentences

,columnist,file_date,text
3,Mike Hulett,2020-04-11,"kamala harris, d-calif., and cory booker, d-n...."
4,Mike Hulett,2020-04-11,"biden, of course, is the friendly old politici..."
6,Mike Hulett,2020-04-11,apparently that is deemed a small price for pu...
12,Mike Hulett,2020-04-11,"actually, one significant problem involves for..."
13,Mike Hulett,2020-04-11,shokin has successfully secured a ukraine cour...
...,...,...,...
1674,Mike Hulett,2022-11-05,"since biden took office, over 2.3 million immi..."
1688,Mike Hulett,2022-11-05,retired four-star general jack keane warns us ...
1691,Mike Hulett,2022-11-05,this column has frequently delineated biden’s ...
1692,Mike Hulett,2022-11-05,we know brother jim and son hunter enriched th...


In [32]:
# len(df_vocab.distinct_words.to_list()[0])
hulett = sorted(df_vocab.distinct_words_excl_stop.to_list()[0])



In [60]:
import nltk
from nltk.stem import WordNetLemmatizer

# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()

hulett_lemma = []

for i in range(len(hulett)):
    word = hulett[i]
    lemma = wnl.lemmatize(word)
    hulett_lemma.append(lemma)

hulett_lemma = set(hulett_lemma)

sorted(hulett_lemma)



# # single word lemmatization examples
# list1 = ['kites', 'babies', 'dogs', 'flying', 'smiling',
# 		'driving', 'died', 'tried', 'feet']
# for words in hulett:
# 	print(words + " ---> " + )
	
#> kites ---> kite
#> babies ---> baby
#> dogs ---> dog
#> flying ---> flying
#> smiling ---> smiling
#> driving ---> driving
#> died ---> died
#> tried ---> tried
#> feet ---> foot


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SAND8464\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['abandon',
 'abandoned',
 'abandoning',
 'abc',
 'abetted',
 'abide',
 'abiding',
 'ability',
 'abjectly',
 'able',
 'abnormal',
 'abolish',
 'abolishing',
 'abortion',
 'abound',
 'abraham',
 'abrams',
 'abroad',
 'abrupt',
 'abruptly',
 'absent',
 'absolute',
 'absolutely',
 'abundance',
 'abundant',
 'abuse',
 'academia',
 'academy',
 'acceleration',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accident',
 'accomplish',
 'accomplished',
 'accomplishment',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accredited',
 'accurate',
 'accurately',
 'accusation',
 'accused',
 'accuser',
 'achieve',
 'achieved',
 'achievement',
 'acknowledge',
 'acknowledged',
 'acquire',
 'acquittal',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'active',
 'activist',
 'activity',
 'actor',
 'actual',
 'actually',
 'ad',
 'adam',
 'add',
 'added',
 'addiction',
 'adding',
 'addition',
 'additional',
 'addled',
 'address',
 'addressed',
 'addressin

In [11]:
hl = df_vocab.all_nonstop_words.to_list()[0]
v = vocab_freq(hl, _singularize=True, top_n=10)
v

,freq
word,
biden,223
trump,204
american,151
president,135
democrat,123
america,102
state,80
would,80
person,79


In [12]:
for i in range(len(df)):
    print(stack_string(df['story_punctuation'][i], 50))

,.",".,-,.,.,-.,,',-.,-.,,-..,.,,.,,".",.".",,-.,-
..,?,,.!-,,?,..,.,.,","..,,.,..:",,."?..,,."":":..
..";!

,:",,.,.."!..,,,.:",,.".,.:"(),.".:",,--?"?:",,,?"
,.,.,."".,.","..-".".,-...,..,-:"...-."!,-""..

;,..!,./.,,.%..:,,,,,,;.,,,..,....!.,....,,.,..,,,
.-..,,,.....,,.

-.,,....,-.,,-.,.-.-.","-?.-"".-"",-.."."-,"".,:"?
":".",.?-?:.""..,.,.,,,,!.-?:--,".",,.,.-,-,.,.().
..,.

.,??..,:,,,-?:",,."??,?,.""."".,,.,,.,.,,-..:,,.(%
)-..,,.,-,(),.(,.)..(%),,,:(%)."".,.:,,--+.,-."".

...,.,,."",,.,,,,.,.,.,-,.,.;....,..,.""."",..,.,,
,.,-.,".",,.,.,,..,.-,-,.:,,?,,""""?.,,?

.,/...,...,.-.,-.??.,.,.?,,.:.......,..?..:",,."--
,..-,..",",."".,:??,--?..

.,,..,.......",",,..,..,..""..",",,,,...,....,-,,-
"."".,.":.","-,,.,,"."-,.:,.-,",",-,.

--....,.:,,,,,,,.,.,,"".,.,,.(--),,,,,,,,,,....,.-
,?!,.?"!"!-"".,.,"""-.",,.,.,.,".":,,.",",..",""."
.!

:,"""".--,,,.--,."".,,",,,.":".",..,."";-.,.?.,--,
.,-."",".":"..",,.,-,,,,"",!-.-,,,,"".-,-".",-.

.,-.,.,,.".""-.",.,,.,,.'.,.,,..,.,%,,.,-.,.,-.